# Cross-task geometry comparison: ODR 1.5 s vs 3.0 s (split-half CV)

Compare Procrustes geometry between the two ODR delay durations
for the **4 common monkeys** (OLI, PIC, ROS, UNI).
All neurons from a monkey are pooled regardless of age.

**Split-half cross-validation:** on each iteration, every monkey’s
neurons are randomly split into two disjoint halves (A, B).
Representations are built per (monkey, split, task), giving
4 monkeys × 2 splits × 2 tasks = 16 entries per iteration.

Using only **cue** (0–500 ms) and **1.5 s delay** (500–2000 ms) epochs.

**Distance categories:**
1. Same monkey, within-task (different splits) — split-half reliability
2. Same monkey, cross-task — cross-task consistency
3. Different monkey, within-task
4. Different monkey, cross-task

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os, sys

sys.path.insert(0, '.')
from functions import (
    load_all_task_data, filter_common_monkeys,
    compute_flat_tuning, cross_task_cv, plot_cross_task,
    print_cross_task_summary,
)

DATA_DIR      = '../data_raw'
N_PCS         = 8
MIN_NEURONS   = N_PCS + 1
BIN_MS        = 50
COMMON_EPOCHS = {'cue': (0, 500), 'delay': (500, 2000)}
N_ITER        = 100
SEED          = 42

## 1. Load data & filter to common monkeys

In [ ]:
task_data = load_all_task_data(DATA_DIR)

odr_tasks, common_monkeys = filter_common_monkeys(task_data,
                                                    task_names=['ODR 1.5s', 'ODR 3.0s'])
print(f'Common monkeys: {common_monkeys}')
for name in ['ODR 1.5s', 'ODR 3.0s']:
    print(f'{name}: {len(odr_tasks[name]["ids"])} neurons')

print('\nNeurons per monkey:')
for mid in common_monkeys:
    n15 = np.sum(odr_tasks['ODR 1.5s']['ids'] == mid)
    n30 = np.sum(odr_tasks['ODR 3.0s']['ids'] == mid)
    print(f'  {mid}: ODR 1.5s = {n15} (splits {n15//2} + {n15 - n15//2}), '
          f'ODR 3.0s = {n30} (splits {n30//2} + {n30 - n30//2})')

## 2. Tuning curves (cue + 1.5 s delay)

In [7]:
task_configs = {
    'ODR 1.5s': dict(data=odr_tasks['ODR 1.5s']['data'], t_range=(-1000, 2500)),
    'ODR 3.0s': dict(data=odr_tasks['ODR 3.0s']['data'], t_range=(-1000, 3500)),
}

tuning_flat = {}
for name, cfg in task_configs.items():
    flat, _, _ = compute_flat_tuning(cfg['data'], cfg['t_range'], COMMON_EPOCHS, BIN_MS)
    tuning_flat[name] = flat
    print(f'{name}: {flat.shape[1]} features')

  neuron 0/391
ODR 1.5s: 16 features
  neuron 0/922
  neuron 500/922
ODR 3.0s: 16 features


## 3. Split-half cross-validation

In [8]:
task_ids = {name: odr_tasks[name]['ids'] for name in odr_tasks}
results = cross_task_cv(tuning_flat, task_ids, N_PCS, MIN_NEURONS, N_ITER, SEED)

  iteration 25/100
  iteration 50/100
  iteration 75/100
  iteration 100/100
Done.


## 4. Results and distance matrix

In [ ]:
plot_cross_task(results)
print_cross_task_summary(results)